In [1]:
# %matplotlib inline

In [2]:
 %matplotlib --list

Available matplotlib backends: ['osx', 'qt4', 'qt5', 'gtk3', 'notebook', 'wx', 'qt', 'nbagg', 'gtk', 'tk', 'inline']


In [9]:
# analyze the focus values for a montage
import cv2
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.colors as mc
import json
import glob
import os
from subprocess import call

min_v_to_plot = 37
#cmap = plt.cm.hot 
cmap = plt.cm.rainbow
scale = 8
trows = 0
tcols = 0
scaled_montage_w = 0
scaled_montage_h = 0
root_dir = ""

def display_surface(img):
    fig=plt.figure()
    ax = Axes3D(fig)
    w,h = img.shape
    xx, yy = np.mgrid[0:w, 0:h]
    
    # plot3D requires a 1D array for x, y, and z
    # ravel() converts the 100x100 array into a 1x10000 array
    #ax.plot_surface(xx,yy,img,  cmap=plt.cm.jet)
    ax.plot_surface(xx,yy,img, rstride=2, cstride=2, linewidth=0, cmap=cmap, vmin=min_v_to_plot, antialiased=False,  shade=True, )
    #ax.plot_wireframe(xx,yy,img, linewidth=1, cmap=cmap)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    fig.add_axes(ax)
    plt.show()

def get_meta_and_montage_files(rootdir):
    '''get the names of the meta and montage files'''
    for name in glob.glob(os.path.join(rootdir, r"_meta*.*")):
        meta = name
    for name in glob.glob(os.path.join(rootdir, r"_montage*.*")):
        montage = name
    return (meta, montage)


def process_one(rootdir):
    global trows, tcols, scaled_montage_h, scaled_montage_w

    meta, montage = get_meta_and_montage_files(rootdir)

    with open(meta) as data_file:
        json_data = json.load(data_file)

    data = json_data[1]['data']

    # figure out the number of rows and columns
    trows = tcols = 0
    for tile in data:
        rp = tile['img_meta']['raster_pos']
        col = rp[0]
        row = rp[1]
        #print row, col
        if row > trows:
            trows = row
        if col > tcols:
            tcols = col

    print trows, tcols

    img = np.empty((int(trows + 1), int(tcols+1)))
    img.fill(min_v_to_plot)

    for tile in data:
        rp = tile['img_meta']['raster_pos']
        col = rp[0]
        row = rp[1]
        focus = tile["focus_score"]
        if focus < min_v_to_plot:  # danger, removing data!!!
            focus = min_v_to_plot
        #print focus
        img[row][col] = focus

    # MONTAGE
    imgmontage = cv2.imread(montage, flags=cv2.IMREAD_GRAYSCALE)
    h, w = imgmontage.shape

    imgmontage = cv2.resize(imgmontage, (w/scale, h/scale) , interpolation = cv2.INTER_AREA)
    scaled_montage_h, scaled_montage_w = imgmontage.shape
    imgmontage = cv2.equalizeHist(imgmontage)
    imgmontage = cv2.cvtColor(imgmontage, cv2.COLOR_GRAY2BGR)

    # Focus values
    img = cv2.resize(img, (scaled_montage_w, scaled_montage_h) , interpolation = cv2.INTER_AREA)
    imin = np.min(img)
    imax = np.max(img)
    idelta = imax - imin

    img2 = np.array((img - imin) / idelta * 255.0, dtype='uint8')

    img = img.astype('uint8')
    print img.shape
    img = cv2.applyColorMap(img2, cv2.COLORMAP_JET)
    print img.shape

    print imgmontage.shape
    print img.shape

    fraction = 0.7
    dst = cv2.addWeighted(imgmontage, fraction, img, 1-fraction, 0)
    cv2.imshow(rootdir, dst)
    cv2.waitKey(-1)


def heatmap_click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        click_row = int(y / float(scaled_montage_h) * (trows+1))
        click_col = int(x / float(scaled_montage_w) * (tcols+1))
        print "row: {}, col: {}".format(click_row, click_col)
        for name in glob.glob(os.path.join(rootdir, r"*_{}_{}.tif").format(click_col, click_row)):
            print name
        if name:
            call([r"C:\Program Files (x86)\IrfanView\i_view32.exe", name])



start = 25 #25
end = 25
for i in range (start, end+1):
    rootdir = r"X:\1x1mm_data\239849_7R_0031_01\0000" + str(i) + r"\0"
    cv2.namedWindow(rootdir)
    cv2.setMouseCallback(rootdir, heatmap_click)
    
    process_one(rootdir)

94 95
(963L, 973L)
(963L, 973L, 3L)
(963L, 973L, 3L)
(963L, 973L, 3L)
row: 15, col: 53
X:\1x1mm_data\239849_7R_0031_01\000025\0\20170308131748272_239849_7R_0031_01_000025_0_53_15.tif
